In [82]:
#--utf-8---
import gzip
import shutil
import pandas as pd
import sys
import io
import json
import re
import time

In [2]:
file = gzip.open('/Users/yurunsong/Desktop/Data/rawCode/python-000000000000.csv.gz') 
pf = pd.read_csv(file, encoding="utf-8")

In [214]:
pf.head(10)

,repo_name,ref,path,copies,content
0,Axford/OpenSCADMachineDesignFramework,refs/heads/master,scripts/static.py,6,#!/usr/bin/env python\n\n# Perform static code...
1,ghandiosm/Test,refs/heads/master,addons/mail/models/mail_followers.py,26,# -*- coding: utf-8 -*-\n\nfrom openerp import...
2,pwarren/AGDeviceControl,refs/heads/master,agdevicecontrol/thirdparty/site-packages/linux...,17,# Copyright (c) 2001-2004 Twisted Matrix Labor...
3,idan/oauthlib,refs/heads/master,tests/oauth2/rfc6749/endpoints/test_metadata.py,1,# -*- coding: utf-8 -*-\nfrom oauthlib.oauth2 ...
4,melviso/beatle,refs/heads/master,beatle/plugin/tools/ast_explorer/res/_param.py,2,"# -*- coding: utf-8 -*-\n\n_param = [\n""16 16 ..."
5,lihf2015/python_exportemail,refs/heads/master,pytest/src/pytest/manage.py,1,#!/usr/bin/env python\nimport os\nimport sys\n...
6,kunallillaney/thunder,refs/heads/master,test/test_extraction_methods_block.py,8,import unittest\n\nfrom thunder import Thunder...
7,40223225/40223225-cdb-0622,refs/heads/master,static/Brython3.1.1-20150328-091302/Lib/unitte...,737,import unittest\nfrom unittest.mock import sen...
8,trangel/OPTpy,refs/heads/master,OPTpy/Abinit/utils.py,1,"from __future__ import print_function, divisio..."
9,Menooker/gem5_pcm,refs/heads/master,src/dev/x86/Cmos.py,69,# Copyright (c) 2008 The Regents of The Univer...


In [256]:
# sum(pf["copies"].tolist())/len(pf["copies"])
# [int(len(pf["copies"])/2)]
# print(pf["content"][pf["copies"]>5000])

In [258]:
# print(pf["content"][142799])

In [215]:
content = pf["content"][7]
print(content)

import unittest
from unittest.mock import sentinel, DEFAULT


class SentinelTest(unittest.TestCase):

    def testSentinels(self):
        self.assertEqual(sentinel.whatever, sentinel.whatever,
                         'sentinel not stored')
        self.assertNotEqual(sentinel.whatever, sentinel.whateverelse,
                            'sentinel should be unique')


    def testSentinelName(self):
        self.assertEqual(str(sentinel.whatever), 'sentinel.whatever',
                         'sentinel name incorrect')


    def testDEFAULT(self):
        self.assertTrue(DEFAULT is sentinel.DEFAULT)

    def testBases(self):
        # If this doesn't raise an AttributeError then help(mock) is broken
        self.assertRaises(AttributeError, lambda: sentinel.__bases__)


if __name__ == '__main__':
    unittest.main()



## Python structure

In [5]:
'''
Generate Tree structure for Python code
'''
class pythonNode():
    def __init__(self, start, end, name, parent, indent):
        self.start =start
        self.end =end
        self.parent =parent
        self.indent =indent
        self.name = name
        self.child = []

In [6]:
startwith = ["from", "import", "class", "def", "for", "while", "if", "else", "elif"]


def restruct(content, start, end, parent, indent):
    subContent = content[start: end]
    pattern = "\n\s{" + str(indent) + "}[a-zA-Z@\'{3}_#0-9]"
    idx = [(m.start(0), m.end(0)) for m in re.finditer(pattern, subContent)]

    name = ""
    for ele in startwith:
        if subContent[2:].startswith(ele):
            name = ele

    if len(idx) == 1:
        return pythonNode(start, end, parent, name, indent)
    else:
        for i in range(len(idx)):
            if i == 0:
                tmp = pythonNode(parent.start, start+idx[0][0], name, parent, indent)  
                restruct(content, parent.start, start + idx[0][0], tmp, indent + 4)
            elif i == len(idx) - 1:
                tmp = pythonNode(start+idx[-1][0]+1, parent.end, name, parent, indent)
                restruct(content, start + idx[-1][0] + 1, parent.end, tmp, indent + 4)
            else:
                tmp = pythonNode(start+idx[i][0]+1,start+idx[i+1][0],name, parent, indent)
                restruct(content, start + idx[i][0] + 1, start + idx[i + 1][0], tmp, indent + 4)
            parent.child.append(tmp)
        return parent

In [7]:
root = pythonNode(0, len(content),"root", None, -1)

In [8]:
restruct(content,0, len(content), root, 0)

In [9]:
# print(len(root.child))
# print(content[root.child[-2].start: root.child[-2].end])

## Python Extract Methods

In [10]:
def searchMethods(content):
    idxs_method = [(m.start(0), m.end(0)) for m in re.finditer(r"(?!\n)\s*def\s.*\(.*\):",content)]
    methodRange = []
    for i in idxs_method:
        subContent = content[i[0]:i[1]]
        indent  = numOfIndent(subContent)
        position = findNextIndent(content[i[1]:],indent)
        methodRange.append((i[0],i[1]+position))
    return methodRange

In [11]:
def numOfIndent(line):
    indent = 0
    while line[indent] == " ":
        indent += 1
    return indent

In [12]:
def findNextIndent(content, indent):
    '''
        find next position where the indent is higher in the content
    '''
    start = 0
    while start < len(content):
        if content[start] == "\n":
            nextIndent = start+1+indent
            if content[nextIndent: nextIndent+2] != "  ":
                return start+1
        start += 1
    return start  

In [13]:
def cleanEachUp(content):
    '''
        Clean up extracted methods
    '''
    indent = numOfIndent(content)
    replacement = "\n"+(" "*indent)
    content = content.replace(replacement, "\n") 
    return content.strip().strip("\n")

In [179]:
'''
    Collect Description and DocString
'''
def cleanUpComments(content):
    content = re.sub(r"\s*#\s.*\n","\n", content)
    content = re.sub(r"[\"]{3}\n\s*.*\n\s*[\"]{3}","", content)
    content = re.sub(r"\n{2,}","\n",content)
    return content


def getDescription(content):
    idx = content.find("\n")
    name = content[:idx]
    descriptions = re.findall(r"(#\s.*\n)+",content)
    docString = re.findall(r"[\"]{3}\n\s*.*\n\s*[\"]{3}",content)
    name = name.replace("def ","")
    name = re.sub("\(.*\):", "", name)
    descriptions = [i.strip("#").strip() for i in descriptions]
    docString = [i.strip("\"").strip("\n").strip() for i in docString]
    return name, descriptions, docString

In [31]:
out = searchMethods(content)

In [61]:
methods = [content[i[0]:i[1]] for i in out]

In [29]:
for i in methods:
    print(cleanEachUp(i))
    print("=====================================================")

def test_nmf(self):
    """
    (BlockMethod) nmf with defaults
    """
    tsc = ThunderContext(self.sc)
    data = tsc.makeExample('sources', dims=(60, 60), centers=[[20, 20], [40, 40]], noise=0.1, seed=42)

    model = SourceExtraction('nmf', componentsPerBlock=1, maxArea=500).fit(data, size=(30, 30))

    assert(model.count == 2)

    # order is irrelevant, but one of these must be true
    ep = 1.0
    cond1 = (model[0].distance([20, 20]) < ep) and (model[1].distance([40, 40]) < ep)
    cond2 = (model[0].distance([40, 40]) < ep) and (model[1].distance([20, 20]) < ep)
    assert(cond1 or cond2)
def test_sima(self):
    """
    (BlockMethod) with SIMA strategy
    """
    # NOTE: this test was brittle and failed non-deterministically with any
    # more than one source
    import sima.segment

    # construct the SIMA strategy
    simaStrategy = sima.segment.STICA(components=1)
    simaStrategy.append(sima.segment.SparseROIsFromMasks(min_size=20))
    simaStrategy.append(sima.segmen

In [42]:
out = getDescription(cleanEachUp(methods[1]))
print(out)
print()
print("Descriptions")
for i in out[1]:
    print(i)
print()
for i in out[2]:
    print(i)
print()
print("======================================")
print(cleanUpComments(cleanEachUp(methods[1])))

('def test_sima(self):', ['NOTE: this test was brittle and failed non-deterministically with any', 'more than one source', 'construct the SIMA strategy', 'create and fit the thunder extraction strategy', 'check that the one center is recovered'], ['(BlockMethod) with SIMA strategy'])

Descriptions
NOTE: this test was brittle and failed non-deterministically with any
more than one source
construct the SIMA strategy
create and fit the thunder extraction strategy
check that the one center is recovered

(BlockMethod) with SIMA strategy

def test_sima(self):
    
    import sima.segment
    simaStrategy = sima.segment.STICA(components=1)
    simaStrategy.append(sima.segment.SparseROIsFromMasks(min_size=20))
    simaStrategy.append(sima.segment.SmoothROIBoundaries())
    simaStrategy.append(sima.segment.MergeOverlapping(threshold=0.5))
    tsc = ThunderContext(self.sc)
    data = tsc.makeExample('sources', dims=(60, 60), centers=[[20, 15]], noise=0.5, seed=42)
    strategy = SourceExtraction

In [180]:
def processing(contents):
    data = []
    for i, body in enumerate(contents):
        if str(body) == "nan":
            continue
        output = searchMethods(body)
        methods = [body[i[0]:i[1]] for i in output]
        for method in methods:
            docs = getDescription(cleanEachUp(method))
            func = cleanUpComments(cleanEachUp(method))
            if len(docs[2]) > 0:
                data.append({
                    "name": docs[0],
                    "descrip": " ".join(docs[1]),
                    "docString": " ".join(docs[2]),
                    "function": func,    
                })
    return data


In [181]:
def store(data):
    with open('data.json', 'w') as fw:
        json.dump(data,fw)

In [188]:
start = time.time()
data = processing(pf["content"])
print(time.time()-start)

200.29402422904968


In [189]:
store(data)

In [191]:
functions = pd.read_json(sys.path[0]+"/data.json", orient='records')

In [210]:
functions

,name,descrip,docString,function
0,elapsedFunc,,1!,def elapsedFunc(self):\n
1,test_nmf,"order is irrelevant, but one of these must be ...",(BlockMethod) nmf with defaults,def test_nmf(self):\n \n tsc = ThunderCo...
2,test_sima,NOTE: this test was brittle and failed non-det...,(BlockMethod) with SIMA strategy,def test_sima(self):\n \n import sima.se...
3,formatter,,register formatters,def formatter(func):\n \n FORMATTERS[get...
4,url,,convert FROM dict OR string TO URL PARAMETERS,def url(value):\n \n return value2url_pa...
...,...,...,...,...
35453,has_poster,,Returns true if a cached poster exists for the...,"def has_poster(self, indexer_id):\n \n p..."
35454,has_banner,,Returns true if a cached banner exists for the...,"def has_banner(self, indexer_id):\n \n b..."
35455,has_fanart,,Returns true if a cached fanart exists for the...,"def has_fanart(self, indexer_id):\n \n f..."
35456,has_poster_thumbnail,,Returns true if a cached poster thumbnail exis...,"def has_poster_thumbnail(self, indexer_id):\n ..."


In [260]:
print(functions["name"][10])
print("--"*10)
print(functions["docString"][10])
print("--"*10)
print(functions["function"][10])

utf82unicode
--------------------
WITH EXPLANATION FOR FAILURE
--------------------
def utf82unicode(value):
    
    try:
        return value.decode("utf8")
    except Exception as e:
        if not _Log:
            _late_import()
